Generate 8x8 image, where each pixel encodes row and column info.

In [1]:
import numpy as np

height = 8
width = 8

pixels = np.zeros(shape=(height, width), dtype='uint16')
for y in range(height):
    for x in range(width):
        pixels[y, x] = ((y % 64) << 6) | x % 64

pixels

array([[  0,   1,   2,   3,   4,   5,   6,   7],
       [ 64,  65,  66,  67,  68,  69,  70,  71],
       [128, 129, 130, 131, 132, 133, 134, 135],
       [192, 193, 194, 195, 196, 197, 198, 199],
       [256, 257, 258, 259, 260, 261, 262, 263],
       [320, 321, 322, 323, 324, 325, 326, 327],
       [384, 385, 386, 387, 388, 389, 390, 391],
       [448, 449, 450, 451, 452, 453, 454, 455]], dtype=uint16)

Divide the image into 4 quadrants.

In [2]:
height = 8
width = 8
quadrants = np.zeros(dtype='uint16', shape=(4, height//2, width//2))
SE, SW, NE, NW = 0, 1, 2, 3

quadrants[SE] = pixels[4:, 4:]
quadrants[SW] = pixels[4:, :4]
quadrants[NE] = pixels[:4, 4:]
quadrants[NW] = pixels[:4, :4]
quadrants

array([[[260, 261, 262, 263],
        [324, 325, 326, 327],
        [388, 389, 390, 391],
        [452, 453, 454, 455]],

       [[256, 257, 258, 259],
        [320, 321, 322, 323],
        [384, 385, 386, 387],
        [448, 449, 450, 451]],

       [[  4,   5,   6,   7],
        [ 68,  69,  70,  71],
        [132, 133, 134, 135],
        [196, 197, 198, 199]],

       [[  0,   1,   2,   3],
        [ 64,  65,  66,  67],
        [128, 129, 130, 131],
        [192, 193, 194, 195]]], dtype=uint16)

Divide the quadrants into octants.

In [3]:
SE0, SE1, SW0, SW1, NE0, NE1, NW0, NW1 = 0, 1, 2, 3, 4, 5, 6, 7
octants = np.zeros(dtype='uint16', shape=(8, height//2//2, width//2))
octants[SE0] = quadrants[SE, 0::2, :]
octants[SE1] = quadrants[SE, 1::2, :]
octants[SW0] = quadrants[SW, 0::2, :]
octants[SW1] = quadrants[SW, 1::2, :]

octants[NE1] = quadrants[NE, 0::2, :]
octants[NE0] = quadrants[NE, 1::2, :]
octants[NW1] = quadrants[NW, 0::2, :]
octants[NW0] = quadrants[NW, 1::2, :]

octants

array([[[260, 261, 262, 263],
        [388, 389, 390, 391]],

       [[324, 325, 326, 327],
        [452, 453, 454, 455]],

       [[256, 257, 258, 259],
        [384, 385, 386, 387]],

       [[320, 321, 322, 323],
        [448, 449, 450, 451]],

       [[ 68,  69,  70,  71],
        [196, 197, 198, 199]],

       [[  4,   5,   6,   7],
        [132, 133, 134, 135]],

       [[ 64,  65,  66,  67],
        [192, 193, 194, 195]],

       [[  0,   1,   2,   3],
        [128, 129, 130, 131]]], dtype=uint16)

North octants stream in reverse order.

In [4]:
octants[NE0] = octants[NE0, ::-1]
octants[NE1] = octants[NE1, ::-1]
octants[NW0] = octants[NW0, ::-1]
octants[NW1] = octants[NW1, ::-1]

octants

array([[[260, 261, 262, 263],
        [388, 389, 390, 391]],

       [[324, 325, 326, 327],
        [452, 453, 454, 455]],

       [[256, 257, 258, 259],
        [384, 385, 386, 387]],

       [[320, 321, 322, 323],
        [448, 449, 450, 451]],

       [[196, 197, 198, 199],
        [ 68,  69,  70,  71]],

       [[132, 133, 134, 135],
        [  4,   5,   6,   7]],

       [[192, 193, 194, 195],
        [ 64,  65,  66,  67]],

       [[128, 129, 130, 131],
        [  0,   1,   2,   3]]], dtype=uint16)

Pack octants to 12 bit pixels.

In [5]:
octant_bytes = np.zeros(shape=(8, 6), dtype='uint16')

for i in range(8):
    pixel_values = octants[i].flatten()

    packed_bytes = 0
    
    for i_pixel, pixel in enumerate(pixel_values):
        pixel = int(pixel)
        packed_bytes |= pixel << (i_pixel * 12)

    
    octant_bytes[i] = np.frombuffer(packed_bytes.to_bytes(12, 'little'), dtype='uint16')
    
octant_bytes

array([[20740,  1552,  4209, 20868, 34328,  6257],
       [20804, 17940,  5233, 20932, 50716,  7281],
       [ 4352,   528,  4145,  4480, 33304,  6193],
       [ 4416, 16916,  5169,  4544, 49692,  7217],
       [20676, 50700,  3184, 20548, 17924,  1136],
       [20612, 34312,  2160, 20484,  1536,   112],
       [ 4288, 49676,  3120,  4160, 16900,  1072],
       [ 4224, 33288,  2096,  4096,   512,    48]], dtype=uint16)

In [6]:
octant_bytes[0].tobytes().hex()

'045110067110845118867118'

In [7]:
int(20740).to_bytes(2, byteorder='little').hex()

'0451'